# chargement des libraries necessaires

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import libraries
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

# Chargement de tous le dataset

## Chargement du train_data

In [ ]:
# This code reads the data from CSV files named "train_data_part_i.csv" for all i from 1 to 10
# and concatenates them into a single pandas DataFrame
train_dataframes = []
for i in tqdm(range(1, 11)):
    train_dataframes.append(pd.read_csv(f'data-train/train_data_part_{i}.csv'))
train_data = pd.concat(train_dataframes, ignore_index=True)

# free up memory by deleting the dataframes we no longer need
del train_dataframes

## Chargement de test_data

In [ ]:
test_data=pd.read_csv('data/test_data.csv')

## Extraction des transactions de 10 000 clients

In [ ]:

import random

# Étape 1 : Obtenez une liste unique des clients dans train_data
unique_clients = train_data['customer_id'].unique()

# Étape 2 : Sélectionnez 10 000 clients aléatoires
sampled_clients = random.sample(list(unique_clients), 10000)

# Étape 3 : Filtrer `final_data` pour ces clients
final_data_sampled = train_data[train_data['customer_id'].isin(sampled_clients)]

# Étape 4 : Filtrer `test_data` pour les mêmes clients
test_data_sampled = test_data[test_data['customer_id'].isin(sampled_clients)]

# Optionnel : Sauvegardez les résultats dans des fichiers CSV
final_data_sampled.to_csv("final_data_sample.csv", index=False)
test_data_sampled.to_csv("test_data_sample.csv", index=False)

# Chargement du jeux de donnees reduit

In [ ]:
final_data=pd.read_csv("final_data_sample.csv")
final_data.head

## Chargement de product_data

In [ ]:
products_data=pd.read_csv('data-train/product_data.csv')

# Pretraitement de product_data

Suprressions des colonnes qui ont plus de 50% de valeurs manquantes

In [ ]:
products_data=products_data.drop(columns=['shelf_level4','ecoscore','salt_reduced','naturality',
                                          'product_description','lactose_free','no_added_salt'])


Creation d'une fonction permettant de remplacer les valeurs manquantes des variables categorielles par le Mode et des variables quantitatives par la median 

In [ ]:

#Imputer les valeur manquantes
import numpy as np

def Imput_ValeurManquate ( Data, columns_Num, columns_Cat, Imput_Num= "median", 
                          Imput_Cat= "Mode" ) :
    for col in columns_Num :
        if Imput_Num == "median" :
            Change = np.median(Data[col].dropna()) 
        else:  
            Change = int(Imput_Num)
        Data[col] = Data[col].fillna(Change)
    for col in columns_Cat :
        if Imput_Cat == "Mode" :
            Change = str(Data[col].mode()[0])
        else:  
            Change = str(Imput_Cat)
        Data[col] = Data[col].fillna(Change)
    return(Data)    
    

# Application de la fonction sur les colonnes 'brand_key','shelf_level3' qui ont prés de 1% de valeurs manquantes
products_data = Imput_ValeurManquate(products_data, ["alcool"], ['brand_key','shelf_level3'])

# Jointure du train_data et de product data

In [ ]:

# Fusion des données sur product_id
final_data = pd.merge(train_data, products_data, on='product_id', how='left')

# Creation des features important du modele

In [ ]:
# Calcule le nombre total de transactions par client
final_data['total_transactions'] = final_data.groupby('customer_id')['transaction_id'].transform('count')

# Calcule le nombre total de produits uniques achetés par client
final_data['total_unique_products'] = final_data.groupby('customer_id')['product_id'].transform('nunique')

# Calcule l'utilisation moyenne de la carte de fidélité par client
# Moyenne des valeurs de la colonne 'has_loyality_card' pour chaque client
final_data['loyalty_card_usage'] = final_data.groupby('customer_id')['has_loyality_card'].transform('mean')

# Calcule le temps écoulé depuis le dernier achat pour chaque client
# Convertit les dates en format datetime, trouve la date la plus récente, puis calcule la différence avec chaque date
final_data['time_since_last_purchase'] = final_data.groupby('customer_id')['date'].transform(
    lambda x: (pd.to_datetime(x).max() - pd.to_datetime(x)).dt.days
)

# Creation de la variable cible pour le modele

In [ ]:
# Chargement des données de test
# On charge le fichier CSV contenant les données de test reduit
test_data = pd.read_csv('data_sample/test_data_sample.csv')

# Création d'un dictionnaire contenant la liste des produits réellement achetés par chaque client
# La clé est 'customer_id' et la valeur est une liste de 'product_id' achetés
actual_purchases = test_data.groupby('customer_id')['product_id'].apply(list).to_dict()

# Ajout de la colonne cible ('purchased_next') pour le modèle
# Cette colonne indique si un produit donné (product_id) a été acheté par un client donné (customer_id)
# Valeur 1 : Le produit a été acheté
# Valeur 0 : Le produit n'a pas été acheté
final_data['purchased_next'] = final_data.apply(
    lambda row: 1 if row['product_id'] in actual_purchases.get(row['customer_id'], []) else 0,
    axis=1
)


In [ ]:
# Conversion de la colonne 'date' en format datetime
# Permet d'extraire facilement des informations temporelles (jour de la semaine, mois, etc.)
final_data['date'] = pd.to_datetime(final_data['date'])

# Extraction du jour de la semaine (0 = lundi, 6 = dimanche)
final_data['day_of_week'] = final_data['date'].dt.dayofweek

# Extraction du mois (1 = janvier, 12 = décembre)
final_data['month'] = final_data['date'].dt.month

# Calcul de la fréquence des achats par client
# Regroupe par 'customer_id' et compte le nombre de transactions uniques ('transaction_id')
customer_frequency = final_data.groupby('customer_id')['transaction_id'].nunique().reset_index()
customer_frequency.rename(columns={'transaction_id': 'purchase_frequency'}, inplace=True)

# Fusionne la fréquence des achats avec les données finales
final_data = final_data.merge(customer_frequency, on='customer_id', how='left')

# Identification du produit préféré par client
# Somme les quantités achetées de chaque produit par client
favorite_products = final_data.groupby(['customer_id', 'product_id'])['quantity'].sum().reset_index()
# Trie par client et par quantité (ordre décroissant)
favorite_products = favorite_products.sort_values(['customer_id', 'quantity'], ascending=[True, False]).groupby('customer_id').head(1)
favorite_products.rename(columns={'product_id': 'favorite_product'}, inplace=True)

# Fusionne les produits préférés avec les données finales
final_data = final_data.merge(favorite_products[['customer_id', 'favorite_product']], on='customer_id', how='left')

# Calcul de la quantité moyenne de produits achetés par transaction pour chaque client
customer_avg_products = final_data.groupby('customer_id')['quantity'].mean().reset_index()
customer_avg_products.rename(columns={'quantity': 'avg_products_per_transaction'}, inplace=True)

# Calcul de la sensibilité aux promotions pour chaque client
# Moyenne de la colonne 'is_promo' par client
customer_promo_sensitivity = final_data.groupby('customer_id')['is_promo'].mean().reset_index()
customer_promo_sensitivity.rename(columns={'is_promo': 'promo_sensitivity'}, inplace=True)

# Identification du dernier produit acheté par chaque client
# Trie les données par date, puis récupère le dernier produit acheté
last_purchase = final_data.sort_values('date').groupby('customer_id')['product_id'].last().reset_index()
last_purchase.rename(columns={'product_id': 'last_product'}, inplace=True)

# Identification de la catégorie préférée de chaque client
# Somme les quantités achetées par catégorie ('department_key') pour chaque client
favorite_category = final_data.groupby(['customer_id', 'department_key'])['quantity'].sum().reset_index()
# Trie par client et par quantité (ordre décroissant)
favorite_category = favorite_category.sort_values(['customer_id', 'quantity'], ascending=[True, False]).groupby('customer_id').head(1)
favorite_category.rename(columns={'department_key': 'favorite_category'}, inplace=True)

# Calcul de la popularité globale des produits
# Somme les quantités achetées pour chaque produit
product_popularity = final_data.groupby('product_id')['quantity'].sum().reset_index()
product_popularity.rename(columns={'quantity': 'global_popularity'}, inplace=True)

# Calcul de la sensibilité aux promotions pour chaque produit
# Moyenne de la colonne 'is_promo' par produit
product_promo_sensitivity = final_data.groupby('product_id')['is_promo'].mean().reset_index()
product_promo_sensitivity.rename(columns={'is_promo': 'promo_sensitivity'}, inplace=True)

# Calcul du nombre d'acheteurs uniques par produit
# Compte le nombre unique de clients ayant acheté chaque produit
product_unique_buyers = final_data.groupby('product_id')['customer_id'].nunique().reset_index()
product_unique_buyers.rename(columns={'customer_id': 'unique_buyers'}, inplace=True)

# Exemple de fusion pour toutes les features clients
# Fusionne les informations calculées sur les clients avec les données finales
final_data = final_data.merge(customer_frequency, on='customer_id', how='left')  # Fréquence d'achat
final_data = final_data.merge(customer_avg_products, on='customer_id', how='left')  # Moyenne des produits achetés par transaction
final_data = final_data.merge(customer_promo_sensitivity, on='customer_id', how='left')  # Sensibilité aux promotions
final_data = final_data.merge(last_purchase, on='customer_id', how='left')  # Dernier produit acheté
# final_data = final_data.merge(favorite_products[['customer_id', 'favorite_product']], on='customer_id', how='left')  # Produit préféré
# final_data = final_data.merge(favorite_category[['customer_id', 'favorite_category']], on='customer_id', how='left')  # Catégorie préférée

# Exemple de fusion pour toutes les features produits
# Fusionne les informations calculées sur les produits avec les données finales
final_data = final_data.merge(product_popularity, on='product_id', how='left')  # Popularité globale
final_data = final_data.merge(product_promo_sensitivity, on='product_id', how='left')  # Sensibilité aux promotions
final_data = final_data.merge(product_unique_buyers, on='product_id', how='left')  # Nombre d'acheteurs uniques


In [ ]:
# Définir les modalités valides pour certaines colonnes
# Chaque clé représente une colonne, et la valeur correspond à une liste des modalités à conserver
valid_modalities = {
    'format': ['DRIVE', 'CLCV'],  # Modalités valides pour la colonne 'format'
    'order_channel': ['WEBSITE', 'MOBILE_APP'],  # Modalités valides pour la colonne 'order_channel'
    'department_key': ['Department_25', 'Department_14', 'Department_10', 'Department_22', 'Department_12'],  # Modalités valides pour 'department_key'
    'sector': ['PGC', 'PRODUITS FRAIS TRANSFORMATION']  # Modalités valides pour la colonne 'sector'
}

# Filtrer les colonnes du DataFrame pour ne conserver que les modalités valides
for column, modalities in valid_modalities.items():  # Itère sur les colonnes et leurs modalités valides
    # Remplace les valeurs qui ne sont pas dans les modalités valides par None
    final_data[column] = final_data[column].apply(lambda x: x if x in modalities else None)

# Encoder les variables catégorielles avec la méthode one-hot encoding
# La méthode pd.get_dummies crée des colonnes binaires pour chaque modalité présente dans les colonnes spécifiées
# Ici, les colonnes correspondant aux clés de 'valid_modalities' sont encodées
# L'argument drop_first=False conserve toutes les modalités pour chaque colonne
final_data = pd.get_dummies(final_data, columns=valid_modalities.keys(), drop_first=False)


In [ ]:
# Identification des types de colonnes à encoder
categorical_cols_onehot = ['day_of_week', 'month']  # Colonnes catégoriques avec un petit nombre de modalités pour One-Hot Encoding
categorical_cols_target = [  # Colonnes catégoriques importantes à encoder via Target Encoding (ou Frequency Encoding)
    'class_key', 'shelf_level1', 'last_product', 'store_id', 'subclass_key', 
    'brand_key', 'shelf_level2', 'shelf_level3'
]
bool_cols = [  # Liste des colonnes booléennes représentant des caractéristiques spécifiques des produits
    'has_loyality_card', 'is_promo', 'bio', 'sugar_free', 'aspartame_free', 'gluten_free', 
    'halal', 'casher', 'eco_friendly', 'local_french', 'artificial_coloring_free', 
    'taste_enhancer_free', 'antibiotic_free', 'reduced_sugar', 'vegetarian', 'pesticide_free', 
    'grain_free', 'no_added_sugar', 'nitrite_free', 'fed_without_ogm', 'no_artificial_flavours', 
    'porc', 'vegan', 'frozen', 'fat_free', 'reduced_fats', 'fresh', 'alcool', 
    'phenylalanine_free', 'palm_oil_free', 'produits_du_monde', 'regional_product', 
    'national_brand', 'first_price_brand', 'carrefour_brand'
]

# Étape 1 : Application du One-Hot Encoding pour les colonnes catégoriques avec peu de modalités
# On transforme les colonnes catégoriques spécifiées dans 'categorical_cols_onehot' en variables binaires
# Exemple : La colonne 'day_of_week' pourrait être transformée en 6 colonnes binaires représentant les jours de la semaine (sauf le premier)
final_data = pd.get_dummies(final_data, columns=categorical_cols_onehot, drop_first=True)

# Définition d'une fonction pour encoder une colonne avec les fréquences
def frequency_encode(df, column):
    # Calcul de la fréquence relative pour chaque modalité de la colonne
    freq_map = df[column].value_counts(normalize=True)  # Renvoie un dictionnaire {valeur: fréquence}
    # Remplacement des valeurs de la colonne par leurs fréquences
    return df[column].map(freq_map)

# Étape 2 : Application du Frequency Encoding sur les colonnes catégoriques ciblées
# On encode chaque colonne dans 'categorical_cols_target' avec les fréquences des modalités
for col in categorical_cols_target:
    final_data[col] = frequency_encode(final_data, col)

# Suppression de la colonne redondante ou incorrecte 'purchase_frequency_y'
# Cela peut être nécessaire si cette colonne a été ajoutée lors de fusion(s) ou calcul(s) précédents
final_data = final_data.drop(columns=['purchase_frequency_y'])


## Sauvegarde du jeu de données pretraité 

In [ ]:
final_data.to_csv('data_sample/final_data.csv',index=False)

# ``DEBUT DE L'ENTRAINEMENT DU MODELE``

In [1]:
import pandas as pd

final_data=pd.read_csv('data_sample/final_data.csv')
final_data

,date,transaction_id,customer_id,product_id,has_loyality_card,store_id,is_promo,quantity,total_transactions,total_unique_products,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2023-07-12,Transaction_1479608,Household_167,Product_8327,0,0.000452,0,1.0,1189,287,...,False,False,False,False,True,False,False,False,False,False
1,2023-07-18,Transaction_993002,Household_167,Product_3846,0,0.000452,0,2.0,1189,287,...,False,False,False,False,True,False,False,False,False,False
2,2023-03-06,Transaction_2113977,Household_1956,Product_70955,0,0.000452,0,1.0,772,596,...,True,False,False,False,False,False,False,False,False,False
3,2022-02-09,Transaction_1686434,Household_1956,Product_204,0,0.000452,0,1.0,772,596,...,False,False,False,False,False,False,False,False,False,False
4,2023-01-23,Transaction_2032633,Household_1956,Product_10212,0,0.000452,1,2.0,772,596,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8867119,2023-06-28,Transaction_703715,Household_72474,Product_56965,0,0.000636,0,2.0,532,170,...,False,False,False,True,False,False,False,False,False,False
8867120,2023-03-19,Transaction_1183149,Household_72474,Product_4110,0,0.000636,0,1.0,532,170,...,True,False,False,False,False,False,False,False,False,False
8867121,2023-02-28,Transaction_8549,Household_72474,Product_58020,0,0.000636,0,1.0,532,170,...,False,False,False,False,False,False,False,False,False,False
8867122,2023-03-30,Transaction_1684683,Household_72474,Product_46541,0,0.000636,0,1.0,532,170,...,True,False,False,False,False,False,False,False,False,False


In [2]:
# Définition d'une fonction pour encoder une colonne avec les fréquences
def frequency_encode(df, column):
    # Calcul de la fréquence relative pour chaque modalité de la colonne
    freq_map = df[column].value_counts(normalize=True)  # Renvoie un dictionnaire {valeur: fréquence}
    # Remplacement des valeurs de la colonne par leurs fréquences
    return df[column].map(freq_map)

In [3]:
# Étape 1 : Calcul de la fréquence d'achat des produits par client
# Le DataFrame `frequency_df` est créé en regroupant par `customer_id` et `product_id`.
# La fréquence est calculée en comptant le nombre de transactions (`transaction_id`) pour chaque paire client-produit.
# La méthode `reset_index` renomme automatiquement la colonne résultante en "frequency".
frequency_df = final_data.groupby(['customer_id', 'product_id'])['transaction_id'].count().reset_index(name='frequency')

# Étape 2 : Fusion des fréquences calculées avec le DataFrame d'origine
# La colonne `frequency` est ajoutée à `final_data` en utilisant une jointure sur les colonnes `customer_id` et `product_id`.
# La jointure est une "left join" pour conserver toutes les lignes existantes dans `final_data`.
final_data = final_data.merge(frequency_df, on=['customer_id', 'product_id'], how='left')

# Étape 3 : Encodage par fréquence pour la colonne `favorite_product`
# La fonction `frequency_encode` est appliquée à la colonne `favorite_product` pour la transformer.
# Chaque modalité de la colonne `favorite_product` est remplacée par sa fréquence relative dans l'ensemble des données.
# Cela permet de capturer l'importance relative des produits préférés des clients.
final_data['favorite_product'] = frequency_encode(final_data, 'favorite_product')


In [4]:
def enhance_data(final_data):
    """
    Ajoute de nouvelles variables pertinentes au DataFrame final_data.

    Paramètre :
        final_data (pd.DataFrame) : Le DataFrame de données initiales.

    Retourne :
        pd.DataFrame : Le DataFrame enrichi avec les nouvelles variables.
    """
    # Conversion explicite de la colonne 'date' au format datetime
    # Cela garantit que la colonne 'date' est correctement formatée pour effectuer des calculs de date.
    if not pd.api.types.is_datetime64_any_dtype(final_data['date']):
        final_data['date'] = pd.to_datetime(final_data['date'], errors='coerce')

    # Calcul de la popularité locale des produits par magasin
    # Compte le nombre de transactions (transaction_id) par magasin et produit pour mesurer leur popularité locale.
    product_popularity_by_store = final_data.groupby(['store_id', 'product_id'])['transaction_id'].count().reset_index()
    product_popularity_by_store.rename(columns={'transaction_id': 'local_popularity'}, inplace=True)
    final_data = final_data.merge(product_popularity_by_store, on=['store_id', 'product_id'], how='left', suffixes=('', '_drop'))

    # Éviter les doublons de colonnes après la jointure
    # Supprime les colonnes inutiles créées lors de la jointure.
    final_data.drop([col for col in final_data.columns if col.endswith('_drop')], axis=1, inplace=True)

    # Calcul du ratio de fréquence globale
    # Fréquence locale divisée par la popularité globale du produit.
    final_data['frequency_global_ratio'] = final_data['frequency'] / final_data['global_popularity']

    # Calcul de la popularité d’un magasin
    # Somme des transactions par magasin pour mesurer son activité globale.
    store_popularity = final_data.groupby('store_id')['transaction_id'].count()
    final_data['store_popularity'] = final_data['store_id'].map(store_popularity)

    # Calcul de la variance des achats mensuels par client
    # Permet de mesurer la régularité des achats mensuels de chaque client.
    monthly_purchases = final_data.groupby(['customer_id', final_data['date'].dt.month])['transaction_id'].count().unstack(fill_value=0)
    final_data['monthly_purchase_variance'] = final_data['customer_id'].map(monthly_purchases.var(axis=1))

    # Calcul du ratio des produits achetés plusieurs fois par client
    # Mesure la fidélité d’un client à certains produits.
    repeat_products = final_data.groupby(['customer_id', 'product_id'])['transaction_id'].count().gt(1).groupby('customer_id').mean()
    final_data['repeat_product_ratio'] = final_data['customer_id'].map(repeat_products)

    # Calcul du temps moyen entre deux transactions pour chaque client
    # Mesure la fréquence des transactions des clients.
    time_diff = final_data.groupby('customer_id')['date'].diff().dt.days
    final_data['avg_time_between_transactions'] = final_data['customer_id'].map(time_diff.groupby(final_data['customer_id']).mean())

    # Calcul de la diversité des catégories achetées par client
    # Mesure la variété des produits achetés par catégorie.
    category_diversity = final_data.groupby('customer_id')['shelf_level1'].nunique()
    final_data['category_diversity'] = final_data['customer_id'].map(category_diversity)

    # Calcul du ratio de fidélité à une marque par client
    # Ratio entre le nombre de marques uniques achetées et le total des marques achetées.
    brand_diversity = final_data.groupby('customer_id')['brand_key'].nunique()
    total_brands = final_data.groupby('customer_id')['brand_key'].count()
    final_data['brand_loyalty_ratio'] = final_data['customer_id'].map(brand_diversity / total_brands)

    # Calcul des ratios pour les produits bio, vegan, et sans gluten
    # Moyenne des indicateurs booléens pour chaque client.
    final_data['bio_ratio'] = final_data.groupby('customer_id')['bio'].transform('mean')
    final_data['vegan_ratio'] = final_data.groupby('customer_id')['vegan'].transform('mean')
    final_data['gluten_free_ratio'] = final_data.groupby('customer_id')['gluten_free'].transform('mean')

    # Calcul du ratio de produits frais par client
    # Moyenne des produits frais achetés.
    fresh_products = final_data.groupby('customer_id')['fresh'].mean()
    final_data['fresh_product_ratio'] = final_data['customer_id'].map(fresh_products)

    # Calcul du ratio de produits locaux par client
    # Moyenne des produits locaux achetés.
    local_ratio = final_data.groupby('customer_id')['local_french'].mean()
    final_data['local_product_ratio'] = final_data['customer_id'].map(local_ratio)

    # Normalisation de la quantité achetée pour une échelle comparable
    # Permet de mettre à l'échelle la variable `quantity`.
    final_data['quantity_norm'] = (final_data['quantity'] - final_data['quantity'].min()) / (final_data['quantity'].max() - final_data['quantity'].min())

    # Calcul de l'indice de sophistication
    # Combinaison pondérée de la quantité et d'une variable binaire liée aux produits de grande consommation (PGC).
    final_data['sophistication_index'] = (
        0.7 * final_data['sector_PGC'] +       # Poids pour les produits PGC
        0.3 * final_data['quantity_norm']      # Poids pour la quantité normalisée
    )

    return final_data
final_data = enhance_data(final_data)


In [5]:
import numpy as np

# Appliquer le logarithme aux variables pertinentes
def apply_log_transformations(data):
    """
    Applique des transformations logarithmiques aux variables continues sélectionnées
    pour réduire l'impact des valeurs extrêmes et améliorer la distribution.

    Paramètres :
        data (pd.DataFrame) : Le DataFrame contenant les données à transformer.

    Retourne :
        pd.DataFrame : Le DataFrame enrichi avec les colonnes transformées logarithmiquement.
    """
    # Variables candidates pour log-transformations
    vars_to_log = [
        'frequency',  # Fréquence d'achat par client pour un produit
        'global_popularity',  # Popularité globale d'un produit
        'store_popularity',  # Popularité d'un magasin
        'avg_time_between_transactions',  # Temps moyen entre transactions d'un client
        'frequency_global_ratio',  # Ratio fréquence d'achat/popularité globale
        'repeat_product_ratio',  # Ratio de produits récurrents achetés par client
        'category_diversity',  # Diversité des catégories achetées par client
        'sophistication_index'  # Indice de sophistication du client basé sur ses achats
    ]
    
    # Vérifier les valeurs non nulles pour éviter log(0)
    for var in vars_to_log:
        if var in data.columns:  # Vérifie si la colonne existe dans le DataFrame
            # Applique la transformation log(1 + x) pour éviter les problèmes avec log(0)
            data[f'log_{var}'] = np.log1p(data[var])
    
    return data

# Appliquer la transformation sur les données
final_data = apply_log_transformations(final_data)


c:\Users\HP\anaconda3\Nouveau_dossier\envs\visualisation\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
# Liste des variables d'origine correspondant aux variables logarithmiques
vars_to_drop = [
    'frequency',
    'global_popularity',
    'store_popularity',
    'avg_time_between_transactions',
    'frequency_global_ratio',
    'repeat_product_ratio',
    'category_diversity',
    'sophistication_index'
]

# Supprimer ces variables
final_data = final_data.drop(columns=vars_to_drop, errors='ignore')

# Les variable utilisée pour notre meilleurs modele

Apres des etudes , nous avons selectionner ces variables

In [7]:
# Liste des variables originales
select_original = [
    'frequency', 'quantity', 'total_transactions',
    'global_popularity', 'unique_buyers',
    'store_id', 'class_key', 'subclass_key', 'brand_key',
    'shelf_level1', 'shelf_level2', 'shelf_level3', 'purchase_frequency_x', 'favorite_product',
    'avg_products_per_transaction', 'promo_sensitivity_x', 'last_product',
    'promo_sensitivity_y', 'loyalty_card_usage', 'total_unique_products', 'time_since_last_purchase',
    "local_popularity", "frequency_global_ratio", "store_popularity", "monthly_purchase_variance",
    "repeat_product_ratio", "avg_time_between_transactions", 'sophistication_index', 'quantity_norm',
    "category_diversity", "brand_loyalty_ratio", "bio_ratio", "vegan_ratio", 
    "gluten_free_ratio", "fresh_product_ratio", "local_product_ratio"
]

# Liste des variables logarithmiques
select_log = [
    'log_frequency', 'log_store_popularity','log_global_popularity',
    'log_avg_time_between_transactions', 'log_frequency_global_ratio',
    'log_repeat_product_ratio', 'log_category_diversity', 'log_sophistication_index'
]

# Créer une liste unique en prenant les logs si disponibles
select = [
    var for var in select_original if var not in [
        'frequency', 'global_popularity', 'store_popularity',
        'avg_time_between_transactions', 'frequency_global_ratio',
        'repeat_product_ratio', 'category_diversity', 'sophistication_index'
    ]
] + select_log

# Afficher la liste finale
select


['quantity',
 'total_transactions',
 'unique_buyers',
 'store_id',
 'class_key',
 'subclass_key',
 'brand_key',
 'shelf_level1',
 'shelf_level2',
 'shelf_level3',
 'purchase_frequency_x',
 'favorite_product',
 'avg_products_per_transaction',
 'promo_sensitivity_x',
 'last_product',
 'promo_sensitivity_y',
 'loyalty_card_usage',
 'total_unique_products',
 'time_since_last_purchase',
 'local_popularity',
 'monthly_purchase_variance',
 'quantity_norm',
 'brand_loyalty_ratio',
 'bio_ratio',
 'vegan_ratio',
 'gluten_free_ratio',
 'fresh_product_ratio',
 'local_product_ratio',
 'log_frequency',
 'log_store_popularity',
 'log_global_popularity',
 'log_avg_time_between_transactions',
 'log_frequency_global_ratio',
 'log_repeat_product_ratio',
 'log_category_diversity',
 'log_sophistication_index']

# Division du jeu de données

In [8]:

# Mise à jour des variables explicatives
X = final_data[select]
y = final_data['purchased_next']

# Diviser en train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


# Entrainement du modele LGBOOST

Apres le RandomizedSearchCV nous avons selectionner ces meilleurs hyperparametre

In [9]:
import lightgbm as lgb
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

# Paramètres fixes pour LightGBM
lgb_params = {
    'n_estimators': 2000,      # Nombre d'arbres
    'learning_rate': 0.1,      # Taux d'apprentissage
    'num_leaves': 60,          # Nombre de feuilles
    'max_depth': -1,           # Profondeur maximale des arbres
    'subsample': 0.8,          # Sous-échantillonnage des données
    'colsample_bytree': 1.0,   # Sous-échantillonnage des colonnes
    'boosting_type': 'gbdt',   # Type de boosting
    'n_jobs': -1,              # Utilisation maximale des cœurs de CPU
    'random_state': 42         # Reproductibilité des résultats
}

# Modèle LightGBM avec les paramètres définis
print('Entraînement du modèle LightGBM...')
lgb_model = lgb.LGBMClassifier(**lgb_params)

# Entraînement du modèle
lgb_model.fit(X_train, y_train)

# Prédictions sur les données de test
lgb_predictions = lgb_model.predict(X_test)

# Rapport de classification
print("LGBoost - Classification Report:\n", classification_report(y_test, lgb_predictions))


Entraînement du modèle LightGBM...
[LightGBM] [Info] Number of positive: 825027, number of negative: 6268672
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.363734 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7573
[LightGBM] [Info] Number of data points in the train set: 7093699, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.116304 -> initscore=-2.027904
[LightGBM] [Info] Start training from score -2.027904
LGBoost - Classification Report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97   1567168
           1       0.98      0.50      0.66    206257

    accuracy                           0.94   1773425
   macro avg       0.96      0.75      0.82   1773425
weighted avg       0.94      0.94      0.93   1773425



# Sauvegarde du modele 

In [10]:
from joblib import dump
dump(lgb_model,'lgb_best2.joblib')

['lgb_best2.joblib']

In [ ]:
from joblib import load
best_lgb=load('lgb_best2.joblib')

# ` LA PREDICTION `

Pour la prediction , j'effecture les meme transformation sur un jeu de donnees train_data afin d'avoir une idee sur hitrate

In [11]:
import pandas as pd
final_data1=pd.read_csv('data-train/train_data_part_5.csv')
products_data = pd.read_csv('data-train/products_data.csv')

C:\Users\HP\AppData\Local\Temp\ipykernel_14712\2033622296.py:3: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  products_data = pd.read_csv('data-train/products_data.csv')


In [12]:

products_data=products_data.drop(columns=['shelf_level4','ecoscore','salt_reduced','naturality',
                                          'product_description','lactose_free','no_added_salt'])

#Imputer les valeur manquantes
import numpy as np

def Imput_ValeurManquate ( Data, columns_Num, columns_Cat, Imput_Num= "median", 
                          Imput_Cat= "Mode" ) :
    for col in columns_Num :
        if Imput_Num == "median" :
            Change = np.median(Data[col].dropna()) 
        else:  
            Change = int(Imput_Num)
        Data[col] = Data[col].fillna(Change)
    for col in columns_Cat :
        if Imput_Cat == "Mode" :
            Change = str(Data[col].mode()[0])
        else:  
            Change = str(Imput_Cat)
        Data[col] = Data[col].fillna(Change)
    return(Data)    
    

products_data = Imput_ValeurManquate(products_data, ["alcool"], ['brand_key','shelf_level3'])

# Fusion des données sur product_id
final_data1 = pd.merge(final_data1, products_data, on='product_id', how='left')

In [13]:
frequency_df1 = final_data1.groupby(['customer_id', 'product_id'])['transaction_id'].count().reset_index(name='frequency')
# Ajouter la colonne fréquence au DataFrame d'origine
final_data1 =final_data1.merge(frequency_df1, on=['customer_id', 'product_id'], how='left')
final_data1['total_transactions'] = final_data1.groupby('customer_id')['transaction_id'].transform('count')
final_data1['total_unique_products'] = final_data1.groupby('customer_id')['product_id'].transform('nunique')


In [14]:
final_data1['date'] = pd.to_datetime(final_data1['date'])
final_data1['day_of_week'] = final_data1['date'].dt.dayofweek
final_data1['month'] = final_data1['date'].dt.month

customer_frequency = final_data1.groupby('customer_id')['transaction_id'].nunique().reset_index()
customer_frequency.rename(columns={'transaction_id': 'purchase_frequency'}, inplace=True)
final_data1 = final_data1.merge(customer_frequency, on='customer_id', how='left')

favorite_products = final_data1.groupby(['customer_id', 'product_id'])['quantity'].sum().reset_index()
favorite_products = favorite_products.sort_values(['customer_id', 'quantity'], ascending=[True, False]).groupby('customer_id').head(1)
favorite_products.rename(columns={'product_id': 'favorite_product'}, inplace=True)
final_data1 = final_data1.merge(favorite_products[['customer_id', 'favorite_product']], on='customer_id', how='left')


customer_avg_products = final_data1.groupby('customer_id')['quantity'].mean().reset_index()
customer_avg_products.rename(columns={'quantity': 'avg_products_per_transaction'}, inplace=True)

customer_promo_sensitivity = final_data1.groupby('customer_id')['is_promo'].mean().reset_index()
customer_promo_sensitivity.rename(columns={'is_promo': 'promo_sensitivity'}, inplace=True)

last_purchase = final_data1.sort_values('date').groupby('customer_id')['product_id'].last().reset_index()
last_purchase.rename(columns={'product_id': 'last_product'}, inplace=True)

favorite_category = final_data1.groupby(['customer_id', 'department_key'])['quantity'].sum().reset_index()
favorite_category = favorite_category.sort_values(['customer_id', 'quantity'], ascending=[True, False]).groupby('customer_id').head(1)
favorite_category.rename(columns={'department_key': 'favorite_category'}, inplace=True)


product_popularity = final_data1.groupby('product_id')['quantity'].sum().reset_index()
product_popularity.rename(columns={'quantity': 'global_popularity'}, inplace=True)


product_promo_sensitivity = final_data1.groupby('product_id')['is_promo'].mean().reset_index()
product_promo_sensitivity.rename(columns={'is_promo': 'promo_sensitivity'}, inplace=True)


product_unique_buyers = final_data1.groupby('product_id')['customer_id'].nunique().reset_index()
product_unique_buyers.rename(columns={'customer_id': 'unique_buyers'}, inplace=True)


# Exemple de fusion pour toutes les features clients
#final_data = train_data.copy()
final_data1 = final_data1.merge(customer_frequency, on='customer_id', how='left')
final_data1 = final_data1.merge(customer_avg_products, on='customer_id', how='left')
final_data1 = final_data1.merge(customer_promo_sensitivity, on='customer_id', how='left')
final_data1 = final_data1.merge(last_purchase, on='customer_id', how='left')
#final_data = final_data.merge(favorite_products[['customer_id', 'favorite_product']], on='customer_id', how='left')
#final_data = final_data.merge(favorite_category[['customer_id', 'favorite_category']], on='customer_id', how='left')


# Exemple de fusion pour toutes les features produits
final_data1 = final_data1.merge(product_popularity, on='product_id', how='left')
final_data1 = final_data1.merge(product_promo_sensitivity, on='product_id', how='left')
final_data1 = final_data1.merge(product_unique_buyers, on='product_id', how='left')


In [15]:
categorical_cols_target = ['class_key','shelf_level1','store_id', 'subclass_key', 'brand_key','shelf_level2', 'shelf_level3']  # Colonnes importantes pour Target Encoding

# Fonction pour encoder avec les fréquences
def frequency_encode(df, column):
    freq_map = df[column].value_counts(normalize=True)  # Fréquences
    return df[column].map(freq_map)

# Encoder chaque colonne catégorique avec les fréquences
for col in categorical_cols_target:
    final_data1[col] = frequency_encode(final_data1, col)

#final_data=final_data.drop(columns=['purchase_frequency_y']

In [16]:
# Modalités à garder
valid_modalities = {
    'format': ['DRIVE', 'CLCV'],
    'order_channel': ['WEBSITE', 'MOBILE_APP'],
    'department_key': ['Department_25', 'Department_14', 'Department_10', 'Department_22', 'Department_12'],
    'sector': ['PGC', 'PRODUITS FRAIS TRANSFORMATION']
}

# Filtrer les colonnes pour ne garder que les modalités pertinentes
for column, modalities in valid_modalities.items():
    final_data1[column] = final_data1[column].apply(lambda x: x if x in modalities else None)

# Encoder avec get_dummies
final_data1 = pd.get_dummies(final_data1, columns=valid_modalities.keys(), drop_first=False)

In [17]:
# Identifiez les colonnes catégoriques et booléennes
categorical_cols_onehot = ['day_of_week', 'month']
# Step 1: One-Hot Encoding pour les colonnes catégoriques avec peu de modalités
final_data1 = pd.get_dummies(final_data1, columns=categorical_cols_onehot, drop_first=True)

In [18]:
final_data1['favorite_product'] = frequency_encode(final_data1, 'favorite_product')
final_data1['last_product'] = frequency_encode(final_data1, 'last_product')
final_data1['favorite_product']=final_data1['favorite_product'].astype('float64')
final_data1['loyalty_card_usage'] = final_data1.groupby('customer_id')['has_loyality_card'].transform('mean')
#train_data['purchase_frequency'] = train_data.groupby('customer_id')['transaction_id'].transform(lambda x: x.nunique() / 2)  # assuming 2 years of data
final_data1['time_since_last_purchase'] = final_data1.groupby('customer_id')['date'].transform(lambda x: (pd.to_datetime(x).max() - pd.to_datetime(x)).dt.days)

In [19]:
def enhance_data(final_data):
    """
    Ajoute de nouvelles variables pertinentes au DataFrame final_data.
    
    Paramètre :
        final_data (pd.DataFrame) : Le DataFrame de données initiales.
    
    Retourne :
        pd.DataFrame : Le DataFrame enrichi avec les nouvelles variables.
    """
    # Conversion explicite de la colonne 'date' au format datetime
    if not pd.api.types.is_datetime64_any_dtype(final_data['date']):
        final_data['date'] = pd.to_datetime(final_data['date'], errors='coerce')

    # Calcul de la popularité locale des produits par magasin
    product_popularity_by_store = final_data.groupby(['store_id', 'product_id'])['transaction_id'].count().reset_index()
    product_popularity_by_store.rename(columns={'transaction_id': 'local_popularity'}, inplace=True)
    final_data = final_data.merge(product_popularity_by_store, on=['store_id', 'product_id'], how='left', suffixes=('', '_drop'))
    
    # Éviter les doublons de colonnes après le merge
    final_data.drop([col for col in final_data.columns if col.endswith('_drop')], axis=1, inplace=True)

    # Calcul du ratio de fréquence globale
    final_data['frequency_global_ratio'] = final_data['frequency'] / final_data['global_popularity']

    # Calcul de la popularité d’un magasin
    store_popularity = final_data.groupby('store_id')['transaction_id'].count()
    final_data['store_popularity'] = final_data['store_id'].map(store_popularity)

    # Calcul de la variance des achats mensuels par client
    monthly_purchases = final_data.groupby(['customer_id', final_data['date'].dt.month])['transaction_id'].count().unstack(fill_value=0)
    final_data['monthly_purchase_variance'] = final_data['customer_id'].map(monthly_purchases.var(axis=1))

    # Calcul du ratio des produits achetés plusieurs fois par client
    repeat_products = final_data.groupby(['customer_id', 'product_id'])['transaction_id'].count().gt(1).groupby('customer_id').mean()
    final_data['repeat_product_ratio'] = final_data['customer_id'].map(repeat_products)

    # Calcul du temps moyen entre deux transactions pour chaque client
    time_diff = final_data.groupby('customer_id')['date'].diff().dt.days
    final_data['avg_time_between_transactions'] = final_data['customer_id'].map(time_diff.groupby(final_data['customer_id']).mean())

    # Calcul de la diversité des catégories achetées par client
    category_diversity = final_data.groupby('customer_id')['shelf_level1'].nunique()
    final_data['category_diversity'] = final_data['customer_id'].map(category_diversity)

    # Calcul du ratio de fidélité à une marque par client
    brand_diversity = final_data.groupby('customer_id')['brand_key'].nunique()
    total_brands = final_data.groupby('customer_id')['brand_key'].count()
    final_data['brand_loyalty_ratio'] = final_data['customer_id'].map(brand_diversity / total_brands)

    # Calcul des ratios pour les produits bio, vegan, et sans gluten
    final_data['bio_ratio'] = final_data.groupby('customer_id')['bio'].transform('mean')
    final_data['vegan_ratio'] = final_data.groupby('customer_id')['vegan'].transform('mean')
    final_data['gluten_free_ratio'] = final_data.groupby('customer_id')['gluten_free'].transform('mean')

    # Calcul du ratio de produits frais par client
    fresh_products = final_data.groupby('customer_id')['fresh'].mean()
    final_data['fresh_product_ratio'] = final_data['customer_id'].map(fresh_products)

    # Calcul du ratio de produits locaux par client
    local_ratio = final_data.groupby('customer_id')['local_french'].mean()
    final_data['local_product_ratio'] = final_data['customer_id'].map(local_ratio)

    # Normalisation de la quantité achetée pour une échelle comparable
    final_data['quantity_norm'] = (final_data['quantity'] - final_data['quantity'].min()) / (final_data['quantity'].max() - final_data['quantity'].min())

    # Calcul de l'indice de sophistication en combinant sector_PGC et quantity avec pondération
    final_data['sophistication_index'] = (
        0.7 * final_data['sector_PGC'] +       # Poids pour la variable sector_PGC (produits PGC)
        0.3 * final_data['quantity_norm']      # Poids plus élevé pour la quantité achetée
    )

    return final_data
final_data1 = enhance_data(final_data1)


In [20]:
import numpy as np

# Appliquer le logarithme aux variables pertinentes
def apply_log_transformations(data):
    # Variables candidates pour log-transformations
    vars_to_log = [
        'frequency',
        'global_popularity',
        'store_popularity',
        'avg_time_between_transactions',
        'frequency_global_ratio',
        'repeat_product_ratio',
        'category_diversity',
        'sophistication_index'
    ]
    
    # Vérifier les valeurs non nulles pour éviter log(0)
    for var in vars_to_log:
        if var in data.columns:
            data[f'log_{var}'] = np.log1p(data[var])  # log1p(x) = log(1 + x), évite log(0)
    
    return data

# Appliquer la transformation sur les données
final_data1 = apply_log_transformations(final_data1)


c:\Users\HP\anaconda3\Nouveau_dossier\envs\visualisation\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\HP\anaconda3\Nouveau_dossier\envs\visualisation\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [21]:
# Liste des variables d'origine correspondant aux variables logarithmiques
vars_to_drop = [
    'frequency',
    'global_popularity',
    'store_popularity',
    'avg_time_between_transactions',
    'frequency_global_ratio',
    'repeat_product_ratio',
    'category_diversity',
    'sophistication_index'
]

# Supprimer ces variables
final_data1 = final_data1.drop(columns=vars_to_drop, errors='ignore')


In [ ]:
# Liste des variables originales
select_original = [
    'frequency', 'quantity', 'total_transactions',
    'global_popularity', 'unique_buyers',
    'store_id', 'class_key', 'subclass_key', 'brand_key',
    'shelf_level1', 'shelf_level2', 'shelf_level3', 'purchase_frequency_x', 'favorite_product',
    'avg_products_per_transaction', 'promo_sensitivity_x', 'last_product',
    'promo_sensitivity_y', 'loyalty_card_usage', 'total_unique_products', 'time_since_last_purchase',
    "local_popularity", "frequency_global_ratio", "store_popularity", "monthly_purchase_variance",
    "repeat_product_ratio", "avg_time_between_transactions", 'sophistication_index', 'quantity_norm',
    "category_diversity", "brand_loyalty_ratio", "bio_ratio", "vegan_ratio", 
    "gluten_free_ratio", "fresh_product_ratio", "local_product_ratio"
]

# Liste des variables logarithmiques
select_log = [
    'log_frequency', 'log_store_popularity','log_global_popularity',
    'log_avg_time_between_transactions', 'log_frequency_global_ratio',
    'log_repeat_product_ratio', 'log_category_diversity', 'log_sophistication_index'
]

# Créer une liste unique en prenant les logs si disponibles
select = [
    var for var in select_original if var not in [
        'frequency', 'global_popularity', 'store_popularity',
        'avg_time_between_transactions', 'frequency_global_ratio',
        'repeat_product_ratio', 'category_diversity', 'sophistication_index'
    ]
] + select_log

# Afficher la liste finale
#select


In [22]:
X1=final_data1[select]

In [23]:
# Prédictions des probabilités d'achat
recommendations_df =final_data1
probabilities = lgb_model.predict_proba(X1)[:, 1]  # Prendre la probabilité de classe positive

# Ajouter les probabilités au DataFrame
recommendations_df['purchase_probability'] = probabilities

In [24]:

# Étape 1: Trier le DataFrame par 'customer_id' et 'purchase_probability'
sorted_recommendations = recommendations_df.sort_values(by=['customer_id', 'purchase_probability'], ascending=[True, False])

# Étape 2: Supprimer les doublons
# Garder uniquement la première occurrence de chaque produit pour chaque client
unique_recommendations = sorted_recommendations.drop_duplicates(subset=['customer_id', 'product_id'])


# Étape 3: Ajouter le rang basé sur la probabilité d'achat
unique_recommendations['rank'] = unique_recommendations.groupby('customer_id')['purchase_probability'].rank(method='first', ascending=False)

# Étape 4: Sélectionner les 10 meilleurs produits pour chaque client
top_recommendations = unique_recommendations[unique_recommendations['rank'] <= 10]

# Afficher les recommandations finales avec le rang
print(top_recommendations[['customer_id', 'product_id', 'purchase_probability', 'rank']])


C:\Users\HP\AppData\Local\Temp\ipykernel_14712\881829423.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_recommendations['rank'] = unique_recommendations.groupby('customer_id')['purchase_probability'].rank(method='first', ascending=False)


             customer_id     product_id  purchase_probability  rank
578741   Household_40001  Product_65763              0.813827   1.0
5091343  Household_40001  Product_28633              0.790623   2.0
6412021  Household_40001  Product_47398              0.739364   3.0
4300600  Household_40001  Product_67559              0.638802   4.0
838820   Household_40001  Product_65000              0.519618   5.0
...                  ...            ...                   ...   ...
5223393  Household_50000  Product_70198              0.404786   6.0
4698150  Household_50000  Product_36150              0.323159   7.0
5488076  Household_50000  Product_70866              0.266328   8.0
2835274  Household_50000  Product_35141              0.233588   9.0
1248787  Household_50000  Product_54454              0.220358  10.0

[100000 rows x 4 columns]


In [25]:
# Afficher les recommandations finales avec le rang
top_recommendations=top_recommendations[['customer_id', 'product_id','rank']]
top_recommendations['rank'] = top_recommendations['rank'].astype(int)
#top_recommendations

In [26]:
# Hitrate@10 evaluation function

def hitrate_at_k(true_data: pd.DataFrame,
                 predicted_data: pd.DataFrame,
                 k: int = 10) -> float:
    """
    This function calculates the hitrate at k for the recommendations.
    It assesses how relevant our 10 product recommendations are.
    In other words, it calculates the proportion of recommended products that are actually purchased by the customer.

    Args:
        true_data: a pandas DataFrame containing the true data
            customer_id: the customer identifier
            product_id: the product identifier that was purchased in the test set
        predicted_data: a pandas DataFrame containing the predicted data
            customer_id: the customer identifier
            product_id: the product identifier that was recommended
            rank: the rank of the recommendation. the rank should be between 1 and 10.
        k: the number of recommendations to consider. k should be between 1 and 10.
    
    Returns:
        The hitrate at k
    """
    
    data = pd.merge(left = true_data, right = predicted_data, how = "left", on = ["customer_id", "product_id"])
    df = data[data["rank"] <= k]
    non_null_counts = df.groupby('customer_id')['rank'].apply(lambda x: x.notna().sum()).reset_index(name='non_null_count')
    distinct_products_per_customer = data.groupby('customer_id')['product_id'].nunique().reset_index(name='distinct_product_count')
    df = pd.merge(left = distinct_products_per_customer, right = non_null_counts, how = "left", on = "customer_id")
    df["denominator"] = [min(df.iloc[i].distinct_product_count,k) for i in range(len(df))]
    df = df.fillna(0)
    return (df["non_null_count"]/df["denominator"]).mean()

In [27]:
test_data=pd.read_csv('data-train/test_data.csv')
# Supposons que 'customer_id' est la clé commune
common_customers = final_data1['customer_id'].unique()  # Extraire les individus uniques de train_data

# Filtrer test_data pour ne garder que les individus communs
test_data = test_data[test_data['customer_id'].isin(common_customers)]

In [28]:
# Calculate the hitrate at k for k = 10
model_hitrate_at_10 = hitrate_at_k(test_data,top_recommendations,10)
print(f"Hitrate@10 est {model_hitrate_at_10:.4f}")

Hitrate@10 est 0.3630
